In [ ]:
import dgl
import torch
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
make_pth = lambda pth: os.sep.join(['..', 'datasets', pth])

In [ ]:
dspth = make_pth('reflacx_densnet225_scan_iou')
dspth

In [ ]:
def get_node(graph, i):
    assert 0 <= i < len(graph.nodes())
    result = {}
    for k in graph.ndata.keys():
        result[k] = graph.ndata[k][i]
    return result

def get_edge(adjmat, i, j):
    assert 0 <= i < adjmat.shape[0]
    assert 0 <= j < adjmat.shape[1]
    return adjmat[i, j]

In [ ]:
ds = dgl.data.CSVDataset(dspth)

In [ ]:
g, labels = ds[0]
g = g.to(torch.device('cuda:0'))

#### Calculating observed area from image

In [ ]:
def mark_area(node, canvas):
    min_x, min_y = [int(a) for a in node['norm_top_left'] * canvas.shape[1]]
    max_x, max_y = [int(a) for a in node['norm_bottom_right'] * canvas.shape[0]]
    canvas[min_x:max_x, min_y:max_y] = 1

In [ ]:
def observed_area_percent(graph, resolution=1000):
    canvas = np.zeros((resolution, resolution))
    for i in range(len(graph.nodes())):
        mark_area(get_node(graph, i), canvas)

    return np.sum(canvas) / (resolution ** 2), canvas

In [ ]:
def dataset_area_percent(dataset, resolution=1000):
    result = {}
    for i, (g, _) in enumerate(dataset):
        result[i], _ = observed_area_percent(g, resolution)
    return result

In [ ]:
dataset_area_percent(ds)